# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - dwalton76's NxNxN Cube Solver</div>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [INFO](#0)
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
  * [FUNCTIONS FROM CRODOC](#3.1)
* [SOLVE](#5)
  * [SCORE](#5.1)
  * [SUBMISSION](#5.2)

<a id="0"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >INFO<br><div> 
    
Some research online led me to GitHub user [dwalton76's rubiks-cube-NxNxN-solver repo](https://github.com/dwalton76/rubiks-cube-NxNxN-solver). I have implemented the NxNxN cube solver below. Note that the solver will skip cubes with a solution state like "N1;N2;N3;..." or "A;B;A;...". The trick I used on the 3x3x3 cubes causes issues when N>=4 (more to come).
    
Special thanks to @wrrosa's Kociemba notebook. This notebook contains modifications of code found [here](https://www.kaggle.com/code/wrrosa/santa-2023-kociemba-s-two-phase-algo-1-116-550)

<a id="1"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >IMPORTS<br><div> 
    


In [1]:
%%bash
git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
cd rubiks-cube-NxNxN-solver
make init

Cloning into 'rubiks-cube-NxNxN-solver'...


rm -rf build dist venv rubikscubennnsolver.egg-info cache ida_search ida_search_via_graph my-pt-states.txt
find . -name __pycache__ | xargs rm -rf
gcc -O3 -o ida_search_via_graph rubikscubennnsolver/ida_search_core.c rubikscubennnsolver/rotate_xxx.c rubikscubennnsolver/ida_search_666.c rubikscubennnsolver/ida_search_777.c rubikscubennnsolver/ida_search_via_graph.c -lm
python3 -m venv venv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Obtaining file:///kaggle/working/rubiks-cube-NxNxN-solver (from -r requirements.dev.txt (line 10))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build wh

In [2]:
%%bash
cd ..
git clone https://github.com/dwalton76/kociemba.git
cd kociemba/kociemba/ckociemba/
make
sudo make install

Cloning into 'kociemba'...


mkdir -p bin
gcc -std=c99 -O3 -D_XOPEN_SOURCE=700 coordcube.c cubiecube.c facecube.c search.c -Iinclude solve.c -o bin/kociemba
cp bin/kociemba /usr/local/bin/


In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

os.chdir('rubiks-cube-NxNxN-solver')
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /kaggle/working/rubiks-cube-NxNxN-solver


<a id="2"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 
    
It is becoming difficult to track down the source of some solutions due to all of the ensemble notebooks in use. I've attempted to locate the sources for the files that give the best solution. The Visualize Using Pythreejs appears to use some solutions from a previous version of this notebook: https://www.kaggle.com/code/zaburo/iterative-replacement-of-k-successive-moves
    
Ensemble sources:

-https://www.kaggle.com/code/yunsuxiaozi/k-step-optimal-solution-santa
-https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards
-https://www.kaggle.com/code/jazivxt/visualize-using-pythreejs
-https://www.kaggle.com/code/seanbearden/kociemba-w-greedy-all-3x3x3-ensemble

In [4]:
sol1 = pd.read_csv('/kaggle/input/k-step-optimal-solution-santa/submission.csv')
sol2 = pd.read_csv('/kaggle/input/bidirectional-brute-force-w-wildcards/submission.csv')
sol3 = pd.read_csv('/kaggle/input/visualize-using-pythreejs/submission.csv')
sol4 = pd.read_csv('/kaggle/input/kociemba-w-greedy-all-3x3x3-ensemble/submission.csv')


ens = pd.DataFrame({
    'solution1': sol1['moves'],
    'solution2': sol2['moves'],
    'solution3': sol3['moves'],
    'solution4': sol4['moves'],
})

ens['ens_solution'] = ens.apply(lambda moves: min(moves, key=lambda x: len(x.split('.'))), axis=1)
ensemble_score = ens['ens_solution'].str.split('.').apply(lambda x: len(x)).sum()

p = '/kaggle/input/santa-2023/'
puzzles = pd.read_csv(p + 'puzzles.csv', index_col='id')
puzzle_info = pd.read_csv(p + 'puzzle_info.csv', index_col='puzzle_type')
submission = pd.read_csv(p + 'sample_submission.csv', index_col='id')

submission['moves'] = ens['ens_solution']

<a id="3"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [5]:
import heapq
import time

def apply_move(state, move):
    # The move vector tells us what index to sample the next state from
    return [state[i] for i in move]

def evaluate_difference(current_state, final_state):
    return (max(0, sum(s != g for s, g in zip(current_state, final_state))))

def evaluate_score(current_state, final_state):
    # Reward having the final position match, and also reward having 2 of the same state adjacent to each other
    # This has to be fast since it's called so often
    return (max(0, sum(s != g for s, g in zip(current_state, final_state))) + sum(s != g for s, g in zip(current_state[1:], current_state[:-1])) + 0.5*sum(s != g for s, g in zip(current_state[2:], current_state[:-2])))


def astar(move_dict, initial_state, final_state, wildcards, max_depth=50, timeout = 180):
    # Priority queue to store nodes with their f-values (g + h)
    start_time = time.time()
    open_set = []
    node_counter = 1
    
    heapq.heappush(open_set, (0, initial_state, []))  # (priority, state, path, remaining wildcards)
    # Set to keep track of visited nodes
    closed_set = set()

    while open_set:
        # Check for timeout
        if time.time() - start_time > timeout:
            print("Timed out.")
            return None, node_counter
        
        # Get the node with the lowest f-value
        current_f, current_state, current_path = heapq.heappop(open_set)
        
        if len(current_path) > max_depth:
            print("Max depth exceeded.")
            return None, node_counter

        diff = evaluate_difference(current_state, final_state)
        if diff <= wildcards:
            # We've achieved our goal. Return the move path.
            return current_path, node_counter

        closed_set.add(tuple(current_state))
        
        for move_str, move in move_dict.items():
            new_state = apply_move(current_state, move)
            if tuple(new_state) not in closed_set:
                if len(current_path) < max_depth:
                    heapq.heappush(open_set, (len(current_path) + 1 + evaluate_difference(new_state, final_state), new_state, current_path + [move_str]))
                    node_counter += 1
    # If no solutions are found:
    print("Open set completed. No solutions.")
    return None, node_counter

In [6]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}

def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]

def Nstate2abcdef(state):
    N_state_split = state.split(';')
    dim = int(np.sqrt(len(N_state_split) // 6))
    dim_2 = dim**2
    
    # create a mapping from N# to A|B|C|D|E|F
    ABCDEF_state_split = ['A'] * dim_2 + ['B'] * dim_2 + ['C'] * dim_2 + ['D'] * dim_2 + ['E'] * dim_2 + ['F'] * dim_2
    NtoABCDEF = {f'N{str(i)}': A for i, A in enumerate(ABCDEF_state_split)}
    
    # map from N-state to ABCDEF-state
    NtoABCDEF_state_split = ';'.join([NtoABCDEF[N] for N in N_state_split])
    return NtoABCDEF_state_split
    

In [7]:
def checkerboard2Nstate(init_state, sol_state, moves, move_pool):

    # show that moving from init using moves results in sol
    state = init_state.split(';')
    for m in moves.split('.'):
        state = move_state(state, m, move_pool)
        
#     print(';'.join(state))
#     print(';'.join(state) == sol_state)
    
    # show that moving from sol using reversed moves results in init
    state = sol_state.split(';')
    rev_moves = [m[1:] if m.startswith('-') else f'-{m}' for m in moves.split('.')[::-1]]
    
#     print(moves.split(';'))
#     print(rev_moves)
    
    for m in rev_moves:
        state = move_state(state, m, move_pool)
        
#     print(';'.join(state))
#     print(';'.join(state) == init_state)
#     print(init_state)
    
    # create an Nstate solution and tranform it with reversed moves
    sol_Nstate = [f'N{i}' for i in range(len(sol_state.split(';')))]
#     print(sol_Nstate)
    
    # create mapping from init_Nstate to colors in init state
    N2Color = {N: C for N, C in zip(sol_Nstate, sol_state.split(';'))}
    
    init_Nstate = sol_Nstate.copy()
    for m in rev_moves:
        init_Nstate = move_state(init_Nstate, m, move_pool)
        
#     print(init_Nstate)
    
    # show that the color mapping fron init_Nstate to init_Cstate matches init_state
    init_Cstate = ';'.join([N2Color[N] for N in init_Nstate])
#     print(init_Cstate)
#     print(init_state)
#     print(init_Cstate == init_state)
    
    return (';'.join(init_Nstate), ';'.join(sol_Nstate), N2Color)
    
    

In [8]:
def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

<a id="3.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">FUNCTIONS FROM CRODOC<br><div>

* https://www.kaggle.com/code/crodoc/1-187-898-greedy-baseline-improvement

In [9]:
allowed_moves = {}

for puzzle_type, row in puzzle_info.iterrows():
    allowed_moves[puzzle_type] = eval(row['allowed_moves'])

def move_state(state, move, moves_pool):
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state

def solve_puzzle(current_puzzle, moves_str):
    # paths.loc[paths['id'] == puzzle_id]
    # todo: assumes index aligned with puzzle id. Need to update.
#     current_puzzle = path.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
#     moves = submission_df.loc[puzzle_id]['moves'].split('.')
    moves = moves_str.split('.')
    moves_pool = allowed_moves[puzzle_type]
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # ignore last state
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx]
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

<a id="5"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 
    
Solver will skip cubes with a solution state like "N1;N2;N3;..." or "A;B;A;...".

In [10]:
# returns single letter face states referring to orientation of center facelets
face_states_4x = {'ABCD': 'A', 'ABDC': 'B', 'ACBD': 'C', 'ACDB': 'D', 'ADBC': 'E', 'ADCB': 'F',
                  'CADB': 'J', 'DACB': 'L', 'BADC': 'H', 'DABC': 'K', 'BACD': 'G', 'CABD': 'I',
                  'DCBA': 'X', 'CDBA': 'V', 'DBCA': 'W', 'BDCA': 'T', 'CBDA': 'U', 'BCDA': 'S',
                  'BDAC': 'N', 'BCAD': 'M', 'CDAB': 'P', 'CBAD': 'O', 'DCAB': 'R', 'DBAC': 'Q'}

# translates numerical facelet ID to alphabetic
N_to_ABCD = {0: 'A', 1: 'B', 4: 'C', 5: 'D'}

def inverse(sequence):
    '''
    Convert a sequence to its inverse
    
    :param sequence: sequence to act upon
    :return: the inverse of the sequence
    '''
    
    return '.'.join([f'{s[1:]}' if s.startswith('-') else f'-{s}' for s in reversed(sequence.split('.'))])

# 4x4x4 manipulations for top face
face_rotations_4x = {'rot_90_clock': '-d3',
                     'rot_90_counter': 'd3',
                     'rot_180': 'd3.d3'}

# 4x4x4 manipulations for 2x2x2 centers based on top face position
face_center_rotations_4x = {# rotates top center counter-clockwise and left face clockwise
                            'rot_90': '-r1.-r2.d1.d2.r1.r2.-d3.-r1.-r2.-d1.-d2.r1.r2.d3', 
                       
                            # rotates top center 180 degrees
                            'rot_180': '-d3.r0.-r3.-d3.-d3.-r0.r3.-d3.r0.-r3.-d3.-d3.-r0.r3'}

# 4x4x4 solutions for the 6 primary states based on top face position
primary_state_solutions_4x = {'a': '', # never used
                              'b': 'r1.f1.-d2.-f1.d2.d2.-r1',
                              'c': '-r1.d2.r1.r2.d2.-r2.-r1.d2.r1',
                              'd': 'f2.-r1.-d1.r1.-f2.d1.-f1.-d1.f1.f2.-d1.-f2',
                              'e': inverse('f2.-r1.-d1.r1.-f2.d1.-f1.-d1.f1.f2.-d1.-f2'), # inverse(d)
                              'f': '-f2.r1.-d2.-r1.d2.d2.f2'}

# 4x4x4 commutators for the secondary states based on top face position
state_commutators_4x = {'A': '', # already solved
                        'B': '',
                        'C': '',
                        'D': '.'.join([primary_state_solutions_4x['f'], face_rotations_4x['rot_90_counter'],
                                       inverse(primary_state_solutions_4x['f']), inverse(face_rotations_4x['rot_90_counter'])]),
                        'E': '.'.join([primary_state_solutions_4x['b'], face_rotations_4x['rot_90_clock'],
                                       inverse(primary_state_solutions_4x['b']), inverse(face_rotations_4x['rot_90_clock'])]),
                        'F': '',
                        'G': '',
                        'H': '.'.join([face_rotations_4x['rot_90_clock'], # H -> P
                                       primary_state_solutions_4x['f'], face_rotations_4x['rot_180'],
                                       inverse(primary_state_solutions_4x['f']), inverse(face_rotations_4x['rot_180']),
                                       inverse(face_rotations_4x['rot_90_clock'])]), 
                        'I': '.'.join([face_center_rotations_4x['rot_180'], # rotate the center to a Q state
                                       inverse(face_rotations_4x['rot_90_clock']), primary_state_solutions_4x['b'], 
                                       face_rotations_4x['rot_90_clock'], inverse(primary_state_solutions_4x['b'])]), # I + 180 rotation -> Q + inverse(U) -> A
                        'J': '',
                        'K': '', 
                        'L': inverse('.'.join([face_center_rotations_4x['rot_180'], # rotate the center to a D state
                                       primary_state_solutions_4x['f'], face_rotations_4x['rot_90_counter'],
                                       inverse(primary_state_solutions_4x['f']), inverse(face_rotations_4x['rot_90_counter'])])),
                        'M': '',
                        'N': '',
                        'O': '',
                        'P': '.'.join([primary_state_solutions_4x['f'], face_rotations_4x['rot_180'],
                                       inverse(primary_state_solutions_4x['f']), inverse(face_rotations_4x['rot_180'])]),
                        'Q': '.'.join([inverse(face_rotations_4x['rot_90_clock']), primary_state_solutions_4x['b'], 
                                               face_rotations_4x['rot_90_clock'], inverse(primary_state_solutions_4x['b'])]),
                        'R': '',
                        'S': '',
                        'T': '.'.join([face_center_rotations_4x['rot_180'], # rotate the center to a D state
                                       primary_state_solutions_4x['f'], face_rotations_4x['rot_90_counter'],
                                       inverse(primary_state_solutions_4x['f']), inverse(face_rotations_4x['rot_90_counter'])]),
                        'U': inverse('.'.join([inverse(face_rotations_4x['rot_90_clock']), primary_state_solutions_4x['b'], 
                                               face_rotations_4x['rot_90_clock'], inverse(primary_state_solutions_4x['b'])])), #inverse(I)
                        'V': '',
                        'W': '',
                        'X': face_center_rotations_4x['rot_180']}


def get_4x_face_state(face):
    '''
    identify the state of the a set of facelets
    
    :param face: a 4x4x4 set of facelets corresponding to a semi-solved face 
                 semi-solved = all correct facelets (colors) but not in correct positions
    :return: the single letter state
    '''
    
    center_tiles = [int(face[i].replace('N', '')) for i in [5, 6, 9, 10]]
    min_val = min(center_tiles)
    center_tiles = [v - min_val for v in center_tiles]
    center_tiles_ABCD = ''.join([N_to_ABCD[v] for v in center_tiles])
    return face_states_4x[center_tiles_ABCD]

##### 
# functions/data for transforming from the top face to another face
####

TtoR_map = {'d0': '-r3', 'd1': '-r2', 'd2': '-r1', 'd3': '-r0',
            '-d0': 'r3', '-d1': 'r2', '-d2': 'r1', '-d3': 'r0',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3',
             
            'r0': '-f3', 'r1': '-f2', 'r2': '-f1', 'r3': '-f0',
            '-r0': 'f3', '-r1': 'f2', '-r2': 'f1', '-r3': 'f0'}

TtoF_map = {'d0': '-f3', 'd1': '-f2', 'd2': '-f1', 'd3': '-f0',
            '-d0': 'f3', '-d1': 'f2', '-d2': 'f1', '-d3': 'f0',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3',
             
            'r0': 'r0', 'r1': 'r1', 'r2': 'r2', 'r3': 'r3',
            '-r0': '-r0', '-r1': '-r1', '-r2': '-r2', '-r3': '-r3'}

TtoL_map = {'d0': 'r0', 'd1': 'r1', 'd2': 'r2', 'd3': 'r3',
            '-d0': '-r0', '-d1': '-r1', '-d2': '-r2', '-d3': '-r3',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3',
             
            'r0': 'f0', 'r1': 'f1', 'r2': 'f2', 'r3': 'f3',
            '-r0': '-f0', '-r1': '-f1', '-r2': '-f2', '-r3': '-f3'}

TtoB_map = {'d0': 'f0', 'd1': 'f1', 'd2': 'f2', 'd3': 'f3',
            '-d0': '-f0', '-d1': '-f1', '-d2': '-f2', '-d3': '-f3',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3',
             
            'r0': '-r3', 'r1': '-r2', 'r2': '-r1', 'r3': '-r0',
            '-r0': 'r3', '-r1': 'r2', '-r2': 'r1', '-r3': 'r0'}

TtoD_map = {'d0': '-d3', 'd1': '-d2', 'd2': '-d1', 'd3': '-d0',
            '-d0': 'd3', '-d1': 'd2', '-d2': 'd1', '-d3': 'd0',
            
            'f0': '-f3', 'f1': '-f2', 'f2': '-f1', 'f3': '-f0',
            '-f0': 'f3', '-f1': 'f2', '-f2': 'f1', '-f3': 'f0',
             
            'r0': 'r0', 'r1': 'r1', 'r2': 'r2', 'r3': 'r3',
            '-r0': '-r0', '-r1': '-r1', '-r2': '-r2', '-r3': '-r3'}

def TtoFace(move, TtoFace_map):
    
    if move not in TtoFace_map:
        newMove = move
    else:
        newMove = f'{TtoFace_map[move]}'
    
    return newMove
        
def transform_sequence_to_face(sequence, target_face_number):
    '''
    Transform a sequence of cube turns from the top face reference to the target face reference
    
    :param face_number: the target face for the transform 
                        0 = top
                        1 = front
                        2 = right
                        3 = back
                        4 = left
                        5 = bottom
    '''
    
    transformed_sequence = ''
    match target_face_number:
        case 0:  # top -> top
            transformed_sequence = sequence
        case 1:   
            transformed_sequence = '.'.join([TtoFace(s, TtoF_map) for s in sequence.split('.')])
        case 2: # top -> right
            transformed_sequence = '.'.join([TtoFace(s, TtoR_map) for s in sequence.split('.')])
        case 3: # top -> back
            transformed_sequence = '.'.join([TtoFace(s, TtoB_map) for s in sequence.split('.')])
        case 4: # top -> left
            transformed_sequence = '.'.join([TtoFace(s, TtoL_map) for s in sequence.split('.')])
        case 5:
            transformed_sequence = '.'.join([TtoFace(s, TtoD_map) for s in sequence.split('.')])
        
    return transformed_sequence

def solve_4x_Nstate(state):
    '''
    Given a 4x4x4 Rubik's cube in a semi-solved state (correct colors but not correct positions)
    
    :param state:  a semi-solved 4x4x4 Rubik's cube
                   semi-solved = all correct facelets (colors) but not in correct positions
    
    '''
    
    # parse all the faces
    new_state = state
    all_moves = []
    
    for face_number in range(6):        
        face_state = get_4x_face_state([new_state[i] for i in range(face_number * 16, face_number * 16 + 16)])
        face_solution_transformed = transform_sequence_to_face(state_commutators_4x[face_state], face_number)
        print(face_number, face_state, face_solution_transformed)
        if face_solution_transformed:
            all_moves.append(face_solution_transformed)
                                               
    return all_moves

In [17]:
# returns single letter face states referring to orientation of center facelets
# same as face_states_4x with more appropriate name
corner_states_5x = {'ABCD': 'A', 'ABDC': 'B', 'ACBD': 'C', 'ACDB': 'D', 'ADBC': 'E', 'ADCB': 'F',
                    'CADB': 'J', 'DACB': 'L', 'BADC': 'H', 'DABC': 'K', 'BACD': 'G', 'CABD': 'I',
                    'DCBA': 'X', 'CDBA': 'V', 'DBCA': 'W', 'BDCA': 'T', 'CBDA': 'U', 'BCDA': 'S',
                    'BDAC': 'N', 'BCAD': 'M', 'CDAB': 'P', 'CBAD': 'O', 'DCAB': 'R', 'DBAC': 'Q'}

edge_states_5x = {'ABCD': 'A', 'ABDC': 'B', 'ACBD': 'C', 'ACDB': 'D', 'ADBC': 'E', 'ADCB': 'F',
                    'CADB': 'J', 'DACB': 'L', 'BADC': 'H', 'DABC': 'K', 'BACD': 'G', 'CABD': 'I',
                    'DCBA': 'X', 'CDBA': 'V', 'DBCA': 'W', 'BDCA': 'T', 'CBDA': 'U', 'BCDA': 'S',
                    'BDAC': 'N', 'BCAD': 'M', 'CDAB': 'P', 'CBAD': 'O', 'DCAB': 'R', 'DBAC': 'Q'}

# translates numerical facelet ID to alphabetic
N_to_ABCD_5x = {0: 'A', 2: 'B', 10: 'C', 12: 'D'}

N_to_ABCE_5x_edge = {0: 'A', 4: 'B', 6: 'C', 10: 'D'}

face_rotations_5x = {'rot_90_clock': '-d4',
                     'rot_90_counter': 'd4',
                     'rot_180': 'd4.d4'}

# 5x5x5 solutions for the 6 primary states based on top face position
primary_state_solutions_5x = {'a': '', # never used
                              'b': 'f2.-d1.-f2.-d1.-r2.d1.r2.-d1.f2.d1.-f2'}

state_commutators_5x_edges = {'A': '', # already solved
                            'B': '',
                            'C': '',
                            'D': '',
                            'E': '',
                            'F': '',
                            'G': '',
                            'H': '', 
                            'I': '',
                            'J': '',
                            'K': '', 
                            'L': '', 
                            'M': '',
                            'N': '',
                            'O': '',
                            'P': '',
                            'Q': '',
                            'R': '',
                            'S': '',
                            'T': '',
                            'U': '',
                            'V': '',
                            'W': '',
                            'X': ''}

##### 
# functions/data for transforming from the top face to another face
####

TtoR_map_5x = {'d0': '-r4', 'd1': '-r3', 'd2': '-r2', 'd3': '-r1', 'd4': '-r0',
            '-d0': 'r4', '-d1': 'r3', '-d2': 'r2', '-d3': 'r1', '-d4': 'r0',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3', 'f4': 'd4',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3', '-f4': '-d4',
             
            'r0': '-f4', 'r1': '-f3', 'r2': '-f2', 'r3': '-f1', 'r4': '-f0',
            '-r0': 'f4', '-r1': 'f3', '-r2': 'f2', '-r3': 'f1', '-r4': 'f0'}

TtoF_map_5x = {'d0': '-f4', 'd1': '-f3', 'd2': '-f2', 'd3': '-f1', 'd4': '-f0',
            '-d0': 'f4', '-d1': 'f3', '-d2': 'f2', '-d3': 'f1', '-d4': 'f0',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3', 'f4': 'd4',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3', '-f4': '-d4',
             
            'r0': 'r0', 'r1': 'r1', 'r2': 'r2', 'r3': 'r3', 'r4': 'r4',
            '-r0': '-r0', '-r1': '-r1', '-r2': '-r2', '-r3': '-r3', '-r4': '-r4'}

TtoL_map_5x = {'d0': 'r0', 'd1': 'r1', 'd2': 'r2', 'd3': 'r3', 'd4': 'r4',
            '-d0': '-r0', '-d1': '-r1', '-d2': '-r2', '-d3': '-r3', '-d4': '-r4',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3', 'f4': 'd4',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3', '-f4': '-d4',
             
            'r0': 'f0', 'r1': 'f1', 'r2': 'f2', 'r3': 'f3', 'r4': 'f4',
            '-r0': '-f0', '-r1': '-f1', '-r2': '-f2', '-r3': '-f3', '-r4': '-f4'}

TtoB_map_5x = {'d0': 'f0', 'd1': 'f1', 'd2': 'f2', 'd3': 'f3', 'd4': 'f4',
            '-d0': '-f0', '-d1': '-f1', '-d2': '-f2', '-d3': '-f3', '-d4': '-f4',
            
            'f0': 'd0', 'f1': 'd1', 'f2': 'd2', 'f3': 'd3', 'f4': 'd4',
            '-f0': '-d0', '-f1': '-d1', '-f2': '-d2', '-f3': '-d3', '-f4': '-d4',
             
            'r0': '-r4', 'r1': '-r3', 'r2': '-r2', 'r3': '-r1', 'r4': '-r0',
            '-r0': 'r4', '-r1': 'r3', '-r2': 'r2', '-r3': 'r1', '-r4': 'r0'}

TtoD_map_5x = {'d0': '-d4', 'd1': '-d3', 'd2': '-d2', 'd3': '-d1', 'd4': '-d0',
            '-d0': 'd4', '-d1': 'd3', '-d2': 'd2', '-d3': 'd1', '-d4': 'd0',
            
            'f0': '-f4', 'f1': '-f3', 'f2': '-f2', 'f3': '-f1', 'f4': '-f0',
            '-f0': 'f4', '-f1': 'f3', '-f2': 'f2', '-f3': 'f1', '-f4': 'f0',
             
            'r0': 'r0', 'r1': 'r1', 'r2': 'r2', 'r3': 'r3', 'r4': 'r4',
            '-r0': '-r0', '-r1': '-r1', '-r2': '-r2', '-r3': '-r3', '-r4': '-r4'}

def TtoFace_5x(move, TtoFace_map):
    
    if move not in TtoFace_map:
        newMove = move
    else:
        newMove = f'{TtoFace_map[move]}'
    
    return newMove
        
def transform_sequence_to_face_5x(sequence, target_face_number):
    '''
    Transform a sequence of cube turns from the top face reference to the target face reference
    
    :param face_number: the target face for the transform 
                        0 = top
                        1 = front
                        2 = right
                        3 = back
                        4 = left
                        5 = bottom
    '''
    
    transformed_sequence = ''
    match target_face_number:
        case 0:  # top -> top
            transformed_sequence = sequence
        case 1:   
            transformed_sequence = '.'.join([TtoFace_5x(s, TtoF_map_5x) for s in sequence.split('.')])
        case 2: # top -> right
            transformed_sequence = '.'.join([TtoFace_5x(s, TtoR_map_5x) for s in sequence.split('.')])
        case 3: # top -> back
            transformed_sequence = '.'.join([TtoFace_5x(s, TtoB_map_5x) for s in sequence.split('.')])
        case 4: # top -> left
            transformed_sequence = '.'.join([TtoFace_5x(s, TtoL_map_5x) for s in sequence.split('.')])
        case 5:
            transformed_sequence = '.'.join([TtoFace_5x(s, TtoD_map_5x) for s in sequence.split('.')])
        
    return transformed_sequence

def get_5x_face_state(face):
    '''
    identify the state of the a set of facelets
    
    :param face: a 5x5x5 set of facelets corresponding to a semi-solved face 
                 semi-solved = all correct facelets (colors) but not in correct positions
    :return: the single letter state
    '''
    
    center_tiles = [int(face[i].replace('N', '')) for i in [6, 8, 16, 18]]
    min_val = min(center_tiles)
#     print(face, center_tiles)
    center_tiles = [v - min_val for v in center_tiles]
#     print(center_tiles)
    center_tiles_ABCD = ''.join([N_to_ABCD_5x[v] for v in center_tiles])
    return corner_states_5x[center_tiles_ABCD]

def get_5x_edge_state(face):
    '''
    identify the state of the a set of edge facelets
    
    :param face: a 5x5x5 set of facelets corresponding to a semi-solved face 
                 semi-solved = all correct facelets (colors) but not in correct positions
    :return: the single letter state
    '''
    
    center_tiles = [int(face[i].replace('N', '')) for i in [7, 11, 13, 17]]
    min_val = min(center_tiles)
#     print(face, center_tiles)
    center_tiles = [v - min_val for v in center_tiles]
#     print(center_tiles)
    center_tiles_ABCD = ''.join([N_to_ABCE_5x_edge[v] for v in center_tiles])
    return edge_states_5x[center_tiles_ABCD]

def transform_4x_to_5x(sequence):
    '''
    transform a 4x sequence of moves to a 5x sequence of moves
    
    :param sequence: a sequence of 4x moves to transform
    
    :return: the 5x sequence of moves
    '''
    
    return sequence.replace('3', '4').replace('2', '3')

def solve_5x_Nstate(state):
    '''
    Given a 5x5x5 Rubik's cube in a semi-solved state (correct colors but not correct positions)
    
    :param state:  a semi-solved 5x5x5x Rubik's cube
                   semi-solved = all correct facelets (colors) but not in correct positions
    
    '''
    
    # parse all the faces
    new_state = state
    all_moves = []
    
    for face_number in range(6):        
        face_state = get_5x_face_state([new_state[i] for i in range(face_number * 25, face_number * 25 + 25)])
        face_solution_transformed = transform_sequence_to_face_5x(state_commutators_4x[face_state], face_number)
        solution_5x = transform_4x_to_5x(face_solution_transformed)
        print(face_number, face_state, solution_5x)
        if face_solution_transformed:
            all_moves.append(face_solution_transformed)
                                               
    return all_moves

def solve_5x_Nstate_edges(state):
    '''
    Given a 5x5x5 Rubik's cube in a semi-solved state (correct colors but not correct positions)
    
    :param state:  a semi-solved 5x5x5x Rubik's cube
                   semi-solved = all correct facelets (colors) but not in correct positions
    :return: sequence of moves to solve the edges
    '''
    
    # parse all the faces
    new_state = state
    all_moves = []
    
    for face_number in range(6):        
        face_state = get_5x_edge_state([new_state[i] for i in range(face_number * 25, face_number * 25 + 25)])
        face_solution_transformed = transform_sequence_to_face_5x(state_commutators_5x_edges[face_state], face_number)
        solution_5x = face_solution_transformed
        print(face_number, face_state, solution_5x)
        if face_solution_transformed:
            all_moves.append(face_solution_transformed)
                                               
    return all_moves

In [12]:
GREEDY_REDUCTION = False

outputs = {}
solution_type = {}
solved_Nstate = []
    
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(240, 241):
            continue
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = move_translation(dim)

        init_state = row['initial_state']
        sol_state = row['solution_state']
        
        # Only attempt to solve cubes with traditional solutions. 
        # That is, cubes that have a solution state where each face 
        # filled with equivilent letters/colors.
        if sol_state[:2*dim**2-1] == ';'.join(['A']*dim**2):
            state = state2ubl(init_state)
            N_state = False
            solution_type[id] = 'standard'
            continue
        else:
            first_face = sol_state.strip().split(';')[:dim**2]
            if first_face == [f'N{i}' for i in range(len(first_face))]:
                solution_type[id] = 'Nstate'
                N_state = True
                if row['puzzle_type'][:6] != 'cube_5':
                    continue
                sol_state = Nstate2abcdef(sol_state)
                init_state = Nstate2abcdef(init_state)
                state = state2ubl(init_state)
            else:
                N_state = False
                solution_type[id] = 'checkerboard'
                continue
                
                # store for later
                init_Cstate = init_state
                sol_Cstate = sol_state
            
                # adjust checkerboard to an Nstate
                init_Nstate, sol_Nstate, N2Colmapping = checkerboard2Nstate(init_state, sol_state, submission.loc[id, 'moves'], moves)
                
                # solve the Nstate
                sol_state = Nstate2abcdef(sol_Nstate)
                init_state = Nstate2abcdef(init_Nstate)
                state = state2ubl(init_state)  
            
        print(f'Starting {id}')
        output = !./rubiks-cube-solver.py --state $state
        outputs[id] = output
        if output[-1][:9] == 'Solution:':
            sol = output[-1].split(': ')[1]
        else:
            for n in range(1, 21):
                if 'Solution:' in output[-n]:
                    sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                    break

        mmoves = '.'.join([M[m] for m in sol.split(' ')])

        new_state = init_state
        for move in mmoves.split('.'):
            new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))

        I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
        manipulations = [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for
                                                                          i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
        for init_moves in manipulations:
            temp_state = new_state
            if len(init_moves) > 0:
                for move in init_moves.split('.'):
                    temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
            if temp_state == sol_state:
                print(f'solved id: {id}')
                if len(init_moves) > 0:
                    mmoves += '.' + init_moves
                break

        # validation
        if N_state:
            state = row['initial_state'].split(";")
        elif solution_type[id] == 'checkerboard':
            state = init_Nstate.split(';')
        for move_name in mmoves.split('.'):
            state = [state[i] for i in moves[move_name]]

        try:
            assert row['solution_state'].split(";") == state

            solved_Nstate.append(id)

            if GREEDY_REDUCTION:
                # reduce moves
                mmoves = solve_puzzle(puzzles.loc[id], mmoves)

            mmoves_length = len(mmoves.split('.'))
            best_moves_length = len(submission.loc[id, 'moves'].split('.')) 

            if mmoves_length < best_moves_length:
                # Insert the moves into the database
                submission.loc[id, 'moves'] = mmoves
        except AssertionError:
            if row['puzzle_type'][:6] == 'cube_5':
                solved_Nstate = solve_5x_Nstate(state)
                stop
                continue
                
            print(f"assertion error for {id}")
            print(np.array(row['solution_state'].split('.')))
            print(';'.join(state))
            continue


Starting 240
solved id: 240
0 L d4.-f3.r1.-d3.-r1.d3.d3.f3.-d4.-f3.-d3.-d3.r1.d3.-r1.f3.-r4.r0.d4.d4.r4.-r0.d4.-r4.r0.d4.d4.r4.-r0.d4
1 U r1.d1.f3.-d1.-f3.-f3.-r1.-f1.r1.f3.f3.d1.-f3.-d1.-r1.f1
2 M 
3 B 
4 C 
5 P f3.r1.d3.-r1.-d3.-d3.-f3.-d1.-d1.f3.d3.d3.r1.-d3.-r1.-f3.d1.d1


NameError: name 'stop' is not defined

In [13]:
# for puzzle 240
# find a solution for corners using 4x4x4 methods
new_state = state

solution = '.'.join([state_commutators_4x['L'], 
                     
                     transform_sequence_to_face(state_commutators_4x['U'], 1),
                     
                     transform_sequence_to_face(face_center_rotations_4x['rot_180'], 2),
                     transform_sequence_to_face(state_commutators_4x['L'], 2),
                     
                     '-r0.-r1.-r2.-r3.-r4',
                     inverse(face_center_rotations_4x['rot_90']),
                     'r0.r1.r2.r3.r4',
                     transform_sequence_to_face(state_commutators_4x['L'], 3),
                     
                     transform_sequence_to_face(state_commutators_4x['P'], 4),
                     
                     transform_sequence_to_face(state_commutators_4x['P'], 5)])
# print(solution)
solution = transform_4x_to_5x(solution)
# print(solution)

for move_name in solution.split('.'):
    new_state = [new_state[i] for i in moves[move_name]]
    
np.array(new_state).reshape( (6, 5, 5) )

array([[['N0', 'N1', 'N2', 'N3', 'N4'],
        ['N5', 'N6', 'N7', 'N8', 'N9'],
        ['N10', 'N11', 'N12', 'N13', 'N14'],
        ['N15', 'N16', 'N17', 'N18', 'N19'],
        ['N20', 'N21', 'N22', 'N23', 'N24']],

       [['N25', 'N26', 'N27', 'N28', 'N29'],
        ['N30', 'N31', 'N38', 'N33', 'N34'],
        ['N35', 'N36', 'N37', 'N32', 'N39'],
        ['N40', 'N41', 'N42', 'N43', 'N44'],
        ['N45', 'N46', 'N47', 'N48', 'N49']],

       [['N50', 'N51', 'N52', 'N53', 'N54'],
        ['N55', 'N56', 'N63', 'N58', 'N59'],
        ['N60', 'N57', 'N62', 'N61', 'N64'],
        ['N65', 'N66', 'N67', 'N68', 'N69'],
        ['N70', 'N71', 'N72', 'N73', 'N74']],

       [['N75', 'N76', 'N77', 'N78', 'N79'],
        ['N80', 'N81', 'N82', 'N83', 'N84'],
        ['N85', 'N92', 'N87', 'N88', 'N89'],
        ['N90', 'N91', 'N86', 'N93', 'N94'],
        ['N95', 'N96', 'N97', 'N98', 'N99']],

       [['N100', 'N101', 'N102', 'N103', 'N104'],
        ['N105', 'N106', 'N117', 'N108', 'N109'],
  

In [20]:
# solve edges
newer_state = new_state
solve_5x_Nstate_edges(newer_state)

solution = '.'.join([#rotate front 90 clockwise
                     transform_sequence_to_face_5x('.'.join([
                         primary_state_solutions_5x['b'], face_rotations_5x['rot_180'],
                         inverse(primary_state_solutions_5x['b']), face_rotations_5x['rot_180']
                     ]), 1)
])

print(solution)
for move_name in solution.split('.'):
    newer_state = [newer_state[i] for i in moves[move_name]]
    
np.array(newer_state).reshape( (6, 5, 5) )

0 A 
1 O 
2 I 
3 F 
4 W 
5 N 
d2.f3.-d2.f3.-r2.-f3.r2.f3.d2.-f3.-d2.-f0.-f0.d2.f3.-d2.-f3.-r2.f3.r2.-f3.d2.-f3.-d2.-f0.-f0


array([[['N0', 'N1', 'N2', 'N3', 'N4'],
        ['N5', 'N6', 'N7', 'N8', 'N9'],
        ['N10', 'N11', 'N12', 'N13', 'N14'],
        ['N15', 'N16', 'N17', 'N18', 'N19'],
        ['N20', 'N21', 'N22', 'N23', 'N24']],

       [['N25', 'N26', 'N27', 'N28', 'N29'],
        ['N30', 'N31', 'N36', 'N33', 'N34'],
        ['N35', 'N38', 'N37', 'N42', 'N39'],
        ['N40', 'N41', 'N32', 'N43', 'N44'],
        ['N45', 'N46', 'N47', 'N48', 'N49']],

       [['N50', 'N51', 'N52', 'N53', 'N54'],
        ['N55', 'N56', 'N63', 'N58', 'N59'],
        ['N60', 'N57', 'N62', 'N61', 'N64'],
        ['N65', 'N66', 'N67', 'N68', 'N69'],
        ['N70', 'N71', 'N72', 'N73', 'N74']],

       [['N75', 'N76', 'N77', 'N78', 'N79'],
        ['N80', 'N81', 'N82', 'N83', 'N84'],
        ['N85', 'N92', 'N87', 'N88', 'N89'],
        ['N90', 'N91', 'N86', 'N93', 'N94'],
        ['N95', 'N96', 'N97', 'N98', 'N99']],

       [['N100', 'N101', 'N102', 'N103', 'N104'],
        ['N105', 'N106', 'N117', 'N108', 'N109'],
  

In [ ]:
# for puzzle 240

# rotate centers
center_rotation = '.'.join([# reorient front to left
                            '-d0.-d1.-d2.-d3',
                            # rotate top clockwise, left rotates counter-clockwise
                            inverse(face_center_rotations_4x['rot_90']),
                            # reorient
                            'd0.d1.d2.d3',
                            '-f0.-f1.-f2.-f3.d0.d0.d1.d1.d2.d2.d3.d3',
                            face_center_rotations_4x['rot_90'],
                            inverse('-f0.-f1.-f2.-f3.d0.d0.d1.d1.d2.d2.d3.d3')
# #                             # rotate top counter-clockwise, J -> A, which rotates left clockwise
#                             face_center_rotations_4x['rot_90'],
#                             # rotate bottom clockwise, K -> D, which rotates left counter-clockwise back to Q
#                             transform_sequence_to_face(inverse(face_center_rotations_4x['rot_90']), 5)
])

# find a solution for centers
new_state = state
# print(np.array(new_state).reshape((6,4,4)))
if center_rotation:
    center_rotation = center_rotation.strip('.')
    while '..' in center_rotation:
        center_rotation.replace('..', '.')
    for move_name in center_rotation.split('.'):
        new_state = [new_state[i] for i in moves[move_name]]
# print(np.array(new_state).reshape((6,4,4)))

center_solution = '.'.join(solve_4x_Nstate(new_state))
if center_solution:
    center_solution = center_solution.strip('.')
    while '..' in center_rotation:
        center_solution.replace('..', '.')   
    for move_name in center_solution.split('.'):
        new_state = [new_state[i] for i in moves[move_name]]
# print(np.array(new_state).reshape((6,4,4)))

# get the complete solution in one form and try greedy optimization
complete_solution = '.'.join([mmoves, center_rotation, center_solution]).replace('..', '.')
reduced_complete_solution = solve_puzzle(puzzles.loc[id], complete_solution)

# test this cube from beginning to end
new_state = row['initial_state'].split(';')
print(np.array(new_state).reshape((6,4,4)))   
for move_name in reduced_complete_solution.split('.'):
    new_state = [new_state[i] for i in moves[move_name]]
print(np.array(new_state).reshape((6,4,4)))   

In [ ]:
#################################

In [ ]:
# check length of old and new
print(f'Old: {len(submission.loc[id, "moves"].split(".")) + 1}\nNew: {len(reduced_complete_solution.split(".")) + 1}')

In [ ]:
# capture the new set of moves
submission.loc[id, 'moves'] = reduced_complete_solution

# this can be saved from SUBMISSION, below

<a id="5.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SCORE<br><div>

In [ ]:
print(f"Score: {submission['moves'].str.split('.').apply(lambda x: len(x)).sum()}")

<a id="5.2"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SUBMISSION<br><div>

In [ ]:
# submission.reset_index().to_csv('submission.csv', index=False)

os.chdir('..')
print("Current Working Directory: ", os.getcwd())

submission.reset_index().to_csv('submission_204solved.csv', index=False)

FYI: There are methods available to reduce some of these solutions further.